## HN-score for each sample

#### Files Required

1. meta-data file
2. HN-ration file

#### basical approach

- Counting per sample by replacing the column name in the file for which the expression ratio is calculated (e.g. SRR22741460 to "shoot") with the sample name listed in the metadata

In [1]:
import datetime
import polars as pl

In [2]:
now = datetime.datetime.now()
print(now)

2025-05-31 07:16:02.083767


In [3]:
# set the threshold
threshold = 5

In [4]:
# polars method: https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.Expr.str.replace_all.html#polars-expr-str-replace-all

metadata = pl.read_csv(
    "../Data/01_Metadata/HS_rice_meta-data.csv",
    separator=","
).with_columns([
    pl.col("Tissue") # remove space
        .str.replace_all(",", " ")  # , to space
        .str.replace_all(r"\s+", " ")  # replace continuous spaces with one space
        .str.replace_all(r"[()]", "")  # remove parentheses (  )
        .str.replace_all(r"\s+$", "")  # remove trailing spaces ('\s+' matches any whitespace character, and '$' matches the end of the string)
        .str.replace_all(" ", "-")  # replace remaining spaces with hyphen
    ]
)
display(metadata.head())

All-pair,Project-pair,Organism,Sub-species,Cultivar,Genotype,Simple stress condition,SRP accession,GEO Accession,Stress,Control,Library_Layout,Stress temperature (day/night) (℃),Control temperature (day/night)(℃),Time,Heat recovery,Treatment condition,Tissue,Period,Instrument,LibrarySelection,DOI,GSM_Pair_name,note,Technical Note
i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,1,"""O.sativa""","""ssp. japonica""","""Lagrue""","""WT""","""HNT_30C_28C_10h""","""SRP413097""","""GSE220996""","""SRR22741460""","""SRR22741464""","""PAIRED""","""30/28""","""30/22.2""","""10h""",null,"""green house""","""seed-endosperm-R6-caryopsis""","""R2 stage (booting) - R6 stage …","""Illumina HiSeq 2000""","""cDNA""","""10.1038/s41598-023-31399-w""",null,"""LaGrue(HNT-tolerant), high-nig…",null
2,2,"""O.sativa""","""ssp. japonica""","""Lagrue""","""WT""","""HNT_30C_28C_10h""","""SRP413097""","""GSE220996""","""SRR22741461""","""SRR22741465""","""PAIRED""","""30/28""","""30/22.2""","""10h""",null,"""green house""","""seed-endosperm-R6-caryopsis""","""R2 stage (booting) - R6 stage …","""Illumina HiSeq 2000""","""cDNA""","""10.1038/s41598-023-31399-w""",null,"""LaGrue(HNT-tolerant), high-nig…",null
3,3,"""O.sativa""","""ssp. japonica""","""Lagrue""","""WT""","""HNT_30C_28C_10h""","""SRP413097""","""GSE220996""","""SRR22741462""","""SRR22741466""","""PAIRED""","""30/28""","""30/22.2""","""10h""",null,"""green house""","""seed-endosperm-R6-caryopsis""","""R2 stage (booting) - R6 stage …","""Illumina HiSeq 2000""","""cDNA""","""10.1038/s41598-023-31399-w""",null,"""LaGrue(HNT-tolerant), high-nig…",null
4,4,"""O.sativa""","""ssp. japonica""","""Lagrue""","""WT""","""HNT_30C_28C_10h""","""SRP413097""","""GSE220996""","""SRR22741463""","""SRR22741467""","""PAIRED""","""30/28""","""30/22.2""","""10h""",null,"""green house""","""seed-endosperm-R6-caryopsis""","""R2 stage (booting) - R6 stage …","""Illumina HiSeq 2000""","""cDNA""","""10.1038/s41598-023-31399-w""",null,"""LaGrue(HNT-tolerant), high-nig…",null
5,5,"""O.sativa""","""ssp. japonica""","""Cypress""","""WT""","""HNT_30C_28C_10h""","""SRP413097""","""GSE220996""","""SRR22741468""","""SRR22741472""","""PAIRED""","""30/28""","""30/22.2""","""10h""",null,"""green house""","""seed-endosperm-R6-caryopsis""","""R2 stage (booting) - R6 stage …","""Illumina HiSeq 2000""","""cDNA""","""10.1038/s41598-023-31399-w""",null,"""Cypress(HNT-sensitive), high-n…",null


In [5]:
HNratio = pl.read_csv(
    "../Data/05_HNratio/HNratio_rice_250524_all.csv", 
    separator=","
)

display(HNratio)

GENEID,SRR22741460,SRR22741461,SRR22741462,SRR22741463,SRR22741468,SRR22741469,SRR22741470,SRR22741471,SRR10991576,SRR10991577,SRR10991578,SRR10991582,SRR10991583,SRR10991584,SRR8140273,SRR8140274,SRR8140275,SRR8140276,SRR8140277,SRR8140278,SRR8140285,SRR8140286,SRR8140287,SRR8140288,SRR8140289,SRR8140290,SRR10423442,SRR10423443,SRR10423444,SRR10423430,SRR10423431,SRR10423432,SRR10423433,SRR10423434,SRR10423435,SRR10423418,…,SRR15060432,SRR15060433,SRR15060434,SRR15060435,SRR15060436,SRR15060437,SRR15060438,SRR15060439,SRR15060440,SRR15060441,SRR15060442,SRR15060443,SRR15060444,SRR15060445,SRR15060446,SRR15060447,SRR15060448,SRR15060449,SRR15060450,SRR15060451,SRR15060452,SRR15060453,SRR15060454,SRR15060455,SRR15060456,SRR22854097,SRR22854098,SRR22854099,SRR22854094,SRR22854095,SRR22854096,SRR23051879,SRR23051878,SRR23051877,SRR23051876,SRR23051875,SRR23051874
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""EPlORYSAT000373610""",1.0,1.0,64.003051,64.326859,0.015755,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,188.995833,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.462188,1.0,1.0,1.0,0.017064,1.0,70.185139,1.284229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373621""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373643""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373795""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373851""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Os12g0641100""",0.330482,0.881209,0.498074,0.510708,0.808446,0.986463,1.23701,0.690745,1.967604,2.090501,1.667619,0.663144,0.915042,1.175697,0.290975,0.287567,0.243554,0.240388,0.361402,0.367039,0.361091,0.361422,0.356863,0.362518,0.45673,0.463282,5.218504,3.569581,3.710993,2.168091,1.36272,1.019475,2.594985,1.684937,1.135803,10.381417,…,1.254025,0.520597,1.459464,0.983362,0.992884,1.093461,1.780748,1.34706,1.17272,2.118824,1.495923,1.029656,0.80882,1.615215,1.139736,0.948756,0.962316,0.721113,1.061525,1.037891,1.287411,2.775175,0.525017,2.270865,0.577094,0.639495,0.560287,0.520431,0.792163,0.735383,0.738452,1.377318,0.974089,0.60175,0.558296,0.972543,1.624633
"""Os12g0641200""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.34581,1.337851,0.722855,0.202596,0.310684,0.482447,2.147339,2.66812,2.470606,2.420584,1.210811,1.68822,1.782365,2.02234,0.864469,0.57654,2.820844,3.900592,1.0,1.0,1.363146,0.224637,2.561818,5.592903,0.891283,1.0,2.886571,0.399646,…,1.626003,0.554418,2.327543,0.708659,1.253904,0.747667,1.21419,1.394378,0.3072,1.

In [6]:
rename_dict = dict(
    zip( # Select a value from the metadata columns that corresponds to the column name in the file containing the expression ratio, and store the corresponding sample name as a dictionary type.
        metadata['Stress'], metadata['Simple stress condition'] # Example: If “SRR22741460” is found in the “Stress” column of the metadata, the corresponding sample is “”seed-endosperm-R6-caryopsis“” in the “Tissue” column.
    )
)

print(rename_dict)

# replaced column name
new_columns = []
seen = set()
for col in HNratio.columns:
    new_col = rename_dict.get(col, col)
    if new_col in seen:
        suffix = 1
        while f"{new_col}_{suffix}" in seen:
            suffix += 1
        new_col = f"{new_col}_{suffix}"
    seen.add(new_col)
    new_columns.append(new_col)

HNratio = HNratio.rename(dict(zip(HNratio.columns, new_columns)))

display(HNratio)

{'SRR22741460': 'HNT_30C_28C_10h', 'SRR22741461': 'HNT_30C_28C_10h', 'SRR22741462': 'HNT_30C_28C_10h', 'SRR22741463': 'HNT_30C_28C_10h', 'SRR22741468': 'HNT_30C_28C_10h', 'SRR22741469': 'HNT_30C_28C_10h', 'SRR22741470': 'HNT_30C_28C_10h', 'SRR22741471': 'HNT_30C_28C_10h', 'SRR10991576': 'HSt_45C_36h', 'SRR10991577': 'HSt_45C_36h', 'SRR10991578': 'HSt_45C_36h', 'SRR10991582': 'HSt_45C_36h', 'SRR10991583': 'HSt_45C_36h', 'SRR10991584': 'HSt_45C_36h', 'SRR8140273': 'HSt_45C_4h', 'SRR8140274': 'HSt_45C_4h', 'SRR8140275': 'HSt_45C_4h', 'SRR8140276': 'HSt_45C_4h', 'SRR8140277': 'HSt_45C_4h', 'SRR8140278': 'HSt_45C_4h', 'SRR8140285': 'HSt_45C_4h', 'SRR8140286': 'HSt_45C_4h', 'SRR8140287': 'HSt_45C_4h', 'SRR8140288': 'HSt_45C_4h', 'SRR8140289': 'HSt_45C_4h', 'SRR8140290': 'HSt_45C_4h', 'SRR10423442': 'HSt_35C_longterm', 'SRR10423443': 'HSt_35C_longterm', 'SRR10423444': 'HSt_35C_longterm', 'SRR10423430': 'HSt_35C_longterm', 'SRR10423431': 'HSt_35C_longterm', 'SRR10423432': 'HSt_35C_longterm', '

GENEID,HNT_30C_28C_10h,HNT_30C_28C_10h_1,HNT_30C_28C_10h_2,HNT_30C_28C_10h_3,HNT_30C_28C_10h_4,HNT_30C_28C_10h_5,HNT_30C_28C_10h_6,HNT_30C_28C_10h_7,HSt_45C_36h,HSt_45C_36h_1,HSt_45C_36h_2,HSt_45C_36h_3,HSt_45C_36h_4,HSt_45C_36h_5,HSt_45C_4h,HSt_45C_4h_1,HSt_45C_4h_2,HSt_45C_4h_3,HSt_45C_4h_4,HSt_45C_4h_5,HSt_45C_4h_6,HSt_45C_4h_7,HSt_45C_4h_8,HSt_45C_4h_9,HSt_45C_4h_10,HSt_45C_4h_11,HSt_35C_longterm,HSt_35C_longterm_1,HSt_35C_longterm_2,HSt_35C_longterm_3,HSt_35C_longterm_4,HSt_35C_longterm_5,HSt_35C_longterm_6,HSt_35C_longterm_7,HSt_35C_longterm_8,HSt_35C_longterm_9,…,HNT_28C_28C_unknown_57,HNT_28C_28C_unknown_58,HNT_28C_28C_unknown_59,HNT_28C_28C_unknown_60,HNT_28C_28C_unknown_61,HNT_28C_28C_unknown_62,HNT_28C_28C_unknown_63,HNT_28C_28C_unknown_64,HNT_28C_28C_unknown_65,HNT_28C_28C_unknown_66,HNT_28C_28C_unknown_67,HNT_28C_28C_unknown_68,HNT_28C_28C_unknown_69,HNT_28C_28C_unknown_70,HNT_28C_28C_unknown_71,HNT_28C_28C_unknown_72,HNT_28C_28C_unknown_73,HNT_28C_28C_unknown_74,HNT_28C_28C_unknown_75,HNT_28C_28C_unknown_76,HNT_28C_28C_unknown_77,HNT_28C_28C_unknown_78,HNT_28C_28C_unknown_79,HNT_28C_28C_unknown_80,HNT_28C_28C_unknown_81,HSh_40C_30min_8,HSh_40C_30min_9,HSh_40C_30min_10,HSh_40C_60min_8,HSh_40C_60min_9,HSh_40C_60min_10,HSt_45C_18h,HSt_45C_18h_1,HSt_45C_18h_2,HSt_45C_18h_3,HSt_45C_18h_4,HSt_45C_18h_5
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""EPlORYSAT000373610""",1.0,1.0,64.003051,64.326859,0.015755,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,188.995833,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.462188,1.0,1.0,1.0,0.017064,1.0,70.185139,1.284229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373621""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373643""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373795""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373851""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Os12g0641100""",0.330482,0.881209,0.498074,0.510708,0.808446,0.986463,1.23701,0.690745,1.967604,2.090501,1.667619,0.663144,0.915042,1.175697,0.290975,0.287567,0.243554,0.240388,0.361402,0.367039,0.361091,0.361422,0.356863,0.362518,0.45673,0.463282,5.218504,3.569581,3.710993,2.168091,1.36272,1.019475,2.594985,1.684937,1.135803,10.381417,…,1.254025,0.520597,1.459464,0.983362,0.992884,1.093461,1.780748,1.34706,1.17272,2.118824,1.495923,1.029656,0.80882,1.615215,1.139736,0.948756,0.962316,0.721113,1.061525,1.037891,1.287411,2.775175,0.525017,2.270865,0.577094,0.639495,0.56

In [7]:
# define a function to categorize the HNratio
def categorize(x):
    if x >= threshold:
        return 'UP'
    elif x <= 1 / threshold:
        return 'DOWN'
    else:
        return 'UNCHANGED'

# apply the categorize function to all columns except GENEID
result = HNratio.with_columns([
    pl.col(col).map_elements(categorize, return_dtype=pl.String)
    for col in HNratio.columns if col != 'GENEID'
])

# display the result
display(result)

GENEID,HNT_30C_28C_10h,HNT_30C_28C_10h_1,HNT_30C_28C_10h_2,HNT_30C_28C_10h_3,HNT_30C_28C_10h_4,HNT_30C_28C_10h_5,HNT_30C_28C_10h_6,HNT_30C_28C_10h_7,HSt_45C_36h,HSt_45C_36h_1,HSt_45C_36h_2,HSt_45C_36h_3,HSt_45C_36h_4,HSt_45C_36h_5,HSt_45C_4h,HSt_45C_4h_1,HSt_45C_4h_2,HSt_45C_4h_3,HSt_45C_4h_4,HSt_45C_4h_5,HSt_45C_4h_6,HSt_45C_4h_7,HSt_45C_4h_8,HSt_45C_4h_9,HSt_45C_4h_10,HSt_45C_4h_11,HSt_35C_longterm,HSt_35C_longterm_1,HSt_35C_longterm_2,HSt_35C_longterm_3,HSt_35C_longterm_4,HSt_35C_longterm_5,HSt_35C_longterm_6,HSt_35C_longterm_7,HSt_35C_longterm_8,HSt_35C_longterm_9,…,HNT_28C_28C_unknown_57,HNT_28C_28C_unknown_58,HNT_28C_28C_unknown_59,HNT_28C_28C_unknown_60,HNT_28C_28C_unknown_61,HNT_28C_28C_unknown_62,HNT_28C_28C_unknown_63,HNT_28C_28C_unknown_64,HNT_28C_28C_unknown_65,HNT_28C_28C_unknown_66,HNT_28C_28C_unknown_67,HNT_28C_28C_unknown_68,HNT_28C_28C_unknown_69,HNT_28C_28C_unknown_70,HNT_28C_28C_unknown_71,HNT_28C_28C_unknown_72,HNT_28C_28C_unknown_73,HNT_28C_28C_unknown_74,HNT_28C_28C_unknown_75,HNT_28C_28C_unknown_76,HNT_28C_28C_unknown_77,HNT_28C_28C_unknown_78,HNT_28C_28C_unknown_79,HNT_28C_28C_unknown_80,HNT_28C_28C_unknown_81,HSh_40C_30min_8,HSh_40C_30min_9,HSh_40C_30min_10,HSh_40C_60min_8,HSh_40C_60min_9,HSh_40C_60min_10,HSt_45C_18h,HSt_45C_18h_1,HSt_45C_18h_2,HSt_45C_18h_3,HSt_45C_18h_4,HSt_45C_18h_5
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""EPlORYSAT000373610""","""UNCHANGED""","""UNCHANGED""","""UP""","""UP""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED"""
"""EPlORYSAT000373621""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED"""
"""EPlORYSAT000373643""",

In [8]:
# remove the suffix from the column name
def remove_suffix(col_name):
    return col_name.rsplit('_', 1)[0]


# get the sample column list
sample_columns = set(remove_suffix(col) for col in result.columns if col != 'GENEID')

print(sample_columns)

counts = []
for sample in sample_columns: # sample_columns means the list of sample names
    # get the dataframe for each sample
    sample_df = result.select(
        ['GENEID'] + [col for col in result.columns if remove_suffix(col) == sample]
    )
    # count the number of UP, DOWN, UNCHANGED
    count = sample_df.with_columns([
        (pl.sum_horizontal([
            (pl.col(col) == "UP").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"UP({sample})"),
        (pl.sum_horizontal([
            (pl.col(col) == "DOWN").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ])).alias(f"DOWN({sample})"),
        pl.sum_horizontal([
            (pl.col(col) == "UNCHANGED").cast(pl.Int32) for col in sample_df.columns if col != "GENEID"
        ]).alias(f"UNCHANGED({sample})")
    ])
    # append the count to the counts list
    counts.append(count)

# combine all samples
final_count = counts[0] # initialize the final_count with the first count
for count in counts[1:]: # [1:] is the list of counts from the second sample to the last sample
    final_count = final_count.join(
        count, 
        on=['GENEID'], 
        how='left',
        coalesce=True
    )

final_count = final_count.select(
    ['GENEID'] + [col for col in final_count.columns if col.startswith('UP(') or col.startswith('DOWN(') or col.startswith('UNCHANGED(')]
)

# (UP,DOWN, UNCHANGED column × sample count ) + create dataframe
display(final_count)

{'HTS_35C_29C_unknown', 'HSh_40C_105min', 'HTS_35C_28C_5days', 'HSh_40C_30min', 'HSh_42C_10min', 'HSh_40C_210min_recov', 'HSt_45C_18h', 'HSh_40C_135min_recov', 'HTS_38C_7h', 'HSh_40C_75min', 'HNT_30C_28C', 'HNT_28C_28C', 'HSh_40C_45min', 'HSt_45C_36h', 'HSh_40C_165min', 'HTS_35C_29C', 'HSh_40C_60min', 'HSt_35C', 'HSh_40C_240min_recov', 'HSh_40C', 'HSh_40C_195min', 'HNT_30C_28C_10h', 'HSh_40C_150min_recov', 'HTS_38C', 'HSh_42C_10min_recov', 'HSh_40C_225min_recov', 'HSh_40C_195min_recov', 'HTS_30C', 'HSh_40C_135min', 'HSh_40C_165min_recov', 'HSh_40C_180min', 'HSh_40C_225min', 'HSh_42C', 'HNT_28C_28C_unknown', 'HSt_45C_4h', 'HSh_40C_120min', 'HTS_35C_28C', 'HSt_45C', 'HSh_40C_240min', 'HSh_40C_15min', 'HSh_40C_150min', 'HSh_40C_180min_recov', 'HSh_40C_90min', 'HTS_30C_18days', 'HSh_40C_210min', 'HSt_35C_longterm'}


GENEID,UP(HTS_35C_29C_unknown),DOWN(HTS_35C_29C_unknown),UNCHANGED(HTS_35C_29C_unknown),UP(HSh_40C_105min),DOWN(HSh_40C_105min),UNCHANGED(HSh_40C_105min),UP(HTS_35C_28C_5days),DOWN(HTS_35C_28C_5days),UNCHANGED(HTS_35C_28C_5days),UP(HSh_40C_30min),DOWN(HSh_40C_30min),UNCHANGED(HSh_40C_30min),UP(HSh_42C_10min),DOWN(HSh_42C_10min),UNCHANGED(HSh_42C_10min),UP(HSh_40C_210min_recov),DOWN(HSh_40C_210min_recov),UNCHANGED(HSh_40C_210min_recov),UP(HSt_45C_18h),DOWN(HSt_45C_18h),UNCHANGED(HSt_45C_18h),UP(HSh_40C_135min_recov),DOWN(HSh_40C_135min_recov),UNCHANGED(HSh_40C_135min_recov),UP(HTS_38C_7h),DOWN(HTS_38C_7h),UNCHANGED(HTS_38C_7h),UP(HSh_40C_75min),DOWN(HSh_40C_75min),UNCHANGED(HSh_40C_75min),UP(HNT_30C_28C),DOWN(HNT_30C_28C),UNCHANGED(HNT_30C_28C),UP(HNT_28C_28C),DOWN(HNT_28C_28C),UNCHANGED(HNT_28C_28C),…,UNCHANGED(HNT_28C_28C_unknown),UP(HSt_45C_4h),DOWN(HSt_45C_4h),UNCHANGED(HSt_45C_4h),UP(HSh_40C_120min),DOWN(HSh_40C_120min),UNCHANGED(HSh_40C_120min),UP(HTS_35C_28C),DOWN(HTS_35C_28C),UNCHANGED(HTS_35C_28C),UP(HSt_45C),DOWN(HSt_45C),UNCHANGED(HSt_45C),UP(HSh_40C_240min),DOWN(HSh_40C_240min),UNCHANGED(HSh_40C_240min),UP(HSh_40C_15min),DOWN(HSh_40C_15min),UNCHANGED(HSh_40C_15min),UP(HSh_40C_150min),DOWN(HSh_40C_150min),UNCHANGED(HSh_40C_150min),UP(HSh_40C_180min_recov),DOWN(HSh_40C_180min_recov),UNCHANGED(HSh_40C_180min_recov),UP(HSh_40C_90min),DOWN(HSh_40C_90min),UNCHANGED(HSh_40C_90min),UP(HTS_30C_18days),DOWN(HTS_30C_18days),UNCHANGED(HTS_30C_18days),UP(HSh_40C_210min),DOWN(HSh_40C_210min),UNCHANGED(HSh_40C_210min),UP(HSt_35C_longterm),DOWN(HSt_35C_longterm),UNCHANGED(HSt_35C_longterm)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,…,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""EPlORYSAT000373610""",3,1,9,1,1,5,0,0,1,1,1,8,1,0,1,0,0,7,0,0,5,0,1,5,0,0,5,1,0,6,0,0,1,0,0,1,…,72,0,0,11,0,0,7,0,1,0,0,0,3,1,0,7,0,1,6,0,0,8,1,0,6,0,0,7,0,0,1,0,1,7,0,0,14
"""EPlORYSAT000373621""",0,0,13,0,0,7,0,0,1,0,0,10,0,0,2,0,0,7,0,0,5,0,0,6,0,0,5,0,0,7,0,0,1,0,0,1,…,81,0,0,11,0,0,7,0,0,1,0,0,3,0,0,8,0,0,7,0,0,8,0,0,7,0,0,7,0,0,1,0,0,8,0,0,14
"""EPlORYSAT000373643""",2,1,10,2,0,5,0,0,1,1,0,9,0,0,2,1,1,5,0,0,5,2,0,4,0,0,5,2,2,3,0,0,1,0,0,1,…,81,0,0,11,2,1,4,0,0,1,0,0,3,3,1,4,2,0,5,2,0,6,0,1,6,2,1,4,0,0,1,2,3,3,0,0,14
"""EPlORYSAT000373795""",1,1,11,0,1,6,0,0,1,0,0,10,0,0,2,0,0,7,0,0,5,0,0,6,0,0,5,1,0,6,0,0,1,0,0,1,…,81,0,0,11,1,1,5,0,0,1,0,0,3,0,0,8,0,1,6,0,0,8,2,0,5,0,1,6,0,0,1,0,0,8,0,0,14
"""EPlORYSAT000373851""",1,2,10,0,1,6,0,0,1,1,0,9,0,0,2,0,0,7,0,0,5,0,1,5,0,0,5,1,1,5,0,0,1,0,0,1,…,81,0,0,11,0,0,7,0,0,1,0,0,3,1,0,7,0,1,6,0,0,8,1,0,6,0,1,6,0,0,1,0,1,7,0,0,14
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Os12g0641100""",0,0,13,1,0,6,0,0,1,0,1,9,0,0,2,0,0,7,0,0,5,0,1,5,1,0,4,1,0,6,0,0,1,0,0,1,…,75,0,0,11,0,0,7,0,0,1,0,0,3,0,0,8,0,0,7,0,0,8,0,0,7,0,0,7,0,0,1,0,1,7,2,0,12
"""Os12g0641200""",0,0,13,0,0,7,0,0,1,0,0,10,0,0,2,0,0,7,2,0,3,0,0,6,2,0,3,0,0,7,0,0,1,0,0,1,…,79,0,0,11,1,0,6,0,0,1,1,0,2,0,0,8,0,0,7,0,0,8,0,0,7,0,2,5,0,0,1,0,0,8,1,1,12
"""Os12g0641300""",0,0,13,1,0,6,0,0,1,0,1,9,0,0,2,0,0,7,0,0,5,0,0,6,1,0,4,0,0,7,0,0,1,0,0,1,…,75,0,0,11,0,0,7,1,0,0,0,0,3,0,0,8,0,1,6,0,0,8,1,0,6,0,0,7,0,0,1,0,1,7,0,2,12


In [12]:
# get the sample name list
sample_names = [col.split('(')[1].split(')')[0] for col in final_count.columns if col.startswith('UP(')]
print(sample_names)

# calculate HNscore for each sample
HNscore_sample_count = final_count.with_columns([
    (
        pl.col(f'UP({sample})') - pl.col(f'DOWN({sample})')
    ).alias(
        f'HN{threshold}({sample})'
    )
    for sample in sample_names # sample_names is the list of sample names
])

# select the columns that start with HN and end with the sample name
HNscore_sample_count = HNscore_sample_count.select(
    ['GENEID'] + 
    [col for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')]
).with_columns( # すべてのサンプルのHNscoreを合計したHNscoreを算出
    pl.sum_horizontal([
        pl.col(col) for col in HNscore_sample_count.columns if col.startswith(f'HN{threshold}(') and col.endswith(')')
    ]).alias(f'HN{threshold}(all)')
).sort(
    by=[f'HN{threshold}(all)'],
    descending=True
)

# save the result to csv
HNscore_sample_count.write_csv(
    f'../Data/06_HNscore_sample/HNscore_rice_stress_condition_count_HN{threshold}.tsv',
    separator='\t'
)

print
display(HNscore_sample_count)

['HTS_35C_29C_unknown', 'HSh_40C_105min', 'HTS_35C_28C_5days', 'HSh_40C_30min', 'HSh_42C_10min', 'HSh_40C_210min_recov', 'HSt_45C_18h', 'HSh_40C_135min_recov', 'HTS_38C_7h', 'HSh_40C_75min', 'HNT_30C_28C', 'HNT_28C_28C', 'HSh_40C_45min', 'HSt_45C_36h', 'HSh_40C_165min', 'HTS_35C_29C', 'HSh_40C_60min', 'HSt_35C', 'HSh_40C_240min_recov', 'HSh_40C', 'HSh_40C_195min', 'HNT_30C_28C_10h', 'HSh_40C_150min_recov', 'HTS_38C', 'HSh_42C_10min_recov', 'HSh_40C_225min_recov', 'HSh_40C_195min_recov', 'HTS_30C', 'HSh_40C_135min', 'HSh_40C_165min_recov', 'HSh_40C_180min', 'HSh_40C_225min', 'HSh_42C', 'HNT_28C_28C_unknown', 'HSt_45C_4h', 'HSh_40C_120min', 'HTS_35C_28C', 'HSt_45C', 'HSh_40C_240min', 'HSh_40C_15min', 'HSh_40C_150min', 'HSh_40C_180min_recov', 'HSh_40C_90min', 'HTS_30C_18days', 'HSh_40C_210min', 'HSt_35C_longterm']


GENEID,HN5(HTS_35C_29C_unknown),HN5(HSh_40C_105min),HN5(HTS_35C_28C_5days),HN5(HSh_40C_30min),HN5(HSh_42C_10min),HN5(HSh_40C_210min_recov),HN5(HSt_45C_18h),HN5(HSh_40C_135min_recov),HN5(HTS_38C_7h),HN5(HSh_40C_75min),HN5(HNT_30C_28C),HN5(HNT_28C_28C),HN5(HSh_40C_45min),HN5(HSt_45C_36h),HN5(HSh_40C_165min),HN5(HTS_35C_29C),HN5(HSh_40C_60min),HN5(HSt_35C),HN5(HSh_40C_240min_recov),HN5(HSh_40C),HN5(HSh_40C_195min),HN5(HNT_30C_28C_10h),HN5(HSh_40C_150min_recov),HN5(HTS_38C),HN5(HSh_42C_10min_recov),HN5(HSh_40C_225min_recov),HN5(HSh_40C_195min_recov),HN5(HTS_30C),HN5(HSh_40C_135min),HN5(HSh_40C_165min_recov),HN5(HSh_40C_180min),HN5(HSh_40C_225min),HN5(HSh_42C),HN5(HNT_28C_28C_unknown),HN5(HSt_45C_4h),HN5(HSh_40C_120min),HN5(HTS_35C_28C),HN5(HSt_45C),HN5(HSh_40C_240min),HN5(HSh_40C_15min),HN5(HSh_40C_150min),HN5(HSh_40C_180min_recov),HN5(HSh_40C_90min),HN5(HTS_30C_18days),HN5(HSh_40C_210min),HN5(HSt_35C_longterm),HN5(all)
str,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32
"""Os04g0107900""",2,7,1,10,2,6,5,6,4,7,0,1,7,5,8,0,10,0,3,15,8,-4,7,1,7,6,7,1,8,7,8,8,1,28,11,7,1,3,7,7,8,7,7,1,7,-3,255
"""Os01g0136100""",3,7,0,10,2,7,5,6,4,7,0,1,7,5,8,0,10,0,7,16,8,-1,7,1,6,7,7,0,8,7,8,8,1,5,11,7,1,3,8,7,8,7,7,1,8,0,245
"""Os02g0259900""",4,7,0,10,2,7,5,6,4,7,0,0,7,5,8,0,10,0,7,15,8,-4,7,1,7,7,7,0,8,7,8,8,1,2,11,7,1,3,8,7,8,7,7,0,8,0,238
"""Os02g0259850""",1,7,0,10,2,7,5,6,4,7,0,0,7,5,8,0,10,0,7,15,8,-4,7,1,7,7,7,0,8,7,8,8,1,3,11,7,1,3,8,7,8,7,7,0,8,1,237
"""Os03g0245800""",5,7,1,10,2,7,5,6,4,7,0,0,7,5,8,0,10,0,7,16,8,-4,7,1,7,7,7,0,8,7,8,8,1,2,11,7,1,3,8,7,8,7,7,0,8,-4,237
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Os01g0952800""",0,-2,0,-4,0,-3,0,-6,-1,-3,0,-1,-3,-2,-4,0,-4,0,-6,-14,-3,0,-3,0,-2,-4,-4,0,-5,-3,-4,-4,0,-55,-4,-3,0,0,-4,-7,-7,-2,-3,1,-4,0,-173
"""Os07g0142100""",0,-2,0,-3,0,-3,-3,-6,-2,-2,0,-1,-3,0,-4,0,-4,0,-6,-14,-3,0,-3,0,0,-4,-4,1,-5,-3,-5,-4,0,-65,0,-3,0,0,-3,-7,-7,-2,-3,1,-4,0,-176
"""Os03g0307200""",1,-3,0,-7,1,-3,0,-6,0,-6,0,-1,-3,0,-4,0,-6,0,-6,-16,-3,-3,-5,0,0,-6,-5,0,-7,-2,-4,-6,1,-43,0,-3,0,0,-7,-7,-8,-2,-6,0,-6,1,-180


&nbsp;

&nbsp;

&nbsp;

## HN-score validation


In [10]:
# select HN5 columns (exclude HN5(all))
hn5_columns = [col for col in HNscore_sample_count.columns if col.startswith("HN5(") and col != "HN5(all)"]
display(hn5_columns)

['HN5(HTS_35C_29C_unknown)',
 'HN5(HSh_40C_105min)',
 'HN5(HTS_35C_28C_5days)',
 'HN5(HSh_40C_30min)',
 'HN5(HSh_42C_10min)',
 'HN5(HSh_40C_210min_recov)',
 'HN5(HSt_45C_18h)',
 'HN5(HSh_40C_135min_recov)',
 'HN5(HTS_38C_7h)',
 'HN5(HSh_40C_75min)',
 'HN5(HNT_30C_28C)',
 'HN5(HNT_28C_28C)',
 'HN5(HSh_40C_45min)',
 'HN5(HSt_45C_36h)',
 'HN5(HSh_40C_165min)',
 'HN5(HTS_35C_29C)',
 'HN5(HSh_40C_60min)',
 'HN5(HSt_35C)',
 'HN5(HSh_40C_240min_recov)',
 'HN5(HSh_40C)',
 'HN5(HSh_40C_195min)',
 'HN5(HNT_30C_28C_10h)',
 'HN5(HSh_40C_150min_recov)',
 'HN5(HTS_38C)',
 'HN5(HSh_42C_10min_recov)',
 'HN5(HSh_40C_225min_recov)',
 'HN5(HSh_40C_195min_recov)',
 'HN5(HTS_30C)',
 'HN5(HSh_40C_135min)',
 'HN5(HSh_40C_165min_recov)',
 'HN5(HSh_40C_180min)',
 'HN5(HSh_40C_225min)',
 'HN5(HSh_42C)',
 'HN5(HNT_28C_28C_unknown)',
 'HN5(HSt_45C_4h)',
 'HN5(HSh_40C_120min)',
 'HN5(HTS_35C_28C)',
 'HN5(HSt_45C)',
 'HN5(HSh_40C_240min)',
 'HN5(HSh_40C_15min)',
 'HN5(HSh_40C_150min)',
 'HN5(HSh_40C_180min_recov)',

In [11]:
# count positive and negative samples
HNscore_sample_validate = HNscore_sample_count.with_columns([
    pl.fold(0, lambda acc, x: acc + (x > 0).cast(pl.Int32), hn5_columns).alias("positive_samples_count"),
    pl.fold(0, lambda acc, x: acc + (x < 0).cast(pl.Int32), hn5_columns).alias("negative_samples_count")
])

# assign tag to the contribution of samples
def tag_contribution(count, type, hn5_column):
    total_samples = len(hn5_column)
    if count == total_samples:
        return f"all samples {type} contribute to the score"
    else:
        return f"{count} / {total_samples} samples {type} contribute to the score"
    
HNscore_sample_validate = HNscore_sample_validate.with_columns(
    [
        pl.lit(len(hn5_columns)).alias("total_samples"),
        pl.col("positive_samples_count").map_elements(lambda x: tag_contribution(x, "positively", hn5_columns), return_dtype=pl.Utf8).alias("positive_contribution"),
        pl.col("negative_samples_count").map_elements(lambda x: tag_contribution(x, "negatively", hn5_columns), return_dtype=pl.Utf8).alias("negative_contribution")
]
).sort(
    by=["HN5(all)"],
    descending=True
).drop(
    hn5_columns
)

# save the result to csv
# HNscore_sample_validate.write_csv(
#     f'../Data/06_HNscore_sample/HNscore_sample_validate_HN{threshold}_rice.tsv',
#     separator='\t'
# )

display(HNscore_sample_validate)

GENEID,HN5(all),positive_samples_count,negative_samples_count,total_samples,positive_contribution,negative_contribution
str,i32,i32,i32,i32,str,str
"""Os04g0107900""",255,41,2,46,"""41 / 46 samples positively con…","""2 / 46 samples negatively cont…"
"""Os01g0136100""",245,39,1,46,"""39 / 46 samples positively con…","""1 / 46 samples negatively cont…"
"""Os02g0259900""",238,37,1,46,"""37 / 46 samples positively con…","""1 / 46 samples negatively cont…"
"""Os02g0259850""",237,38,1,46,"""38 / 46 samples positively con…","""1 / 46 samples negatively cont…"
"""Os03g0245800""",237,38,2,46,"""38 / 46 samples positively con…","""2 / 46 samples negatively cont…"
…,…,…,…,…,…,…
"""Os01g0952800""",-173,1,31,46,"""1 / 46 samples positively cont…","""31 / 46 samples negatively con…"
"""Os07g0142100""",-176,2,29,46,"""2 / 46 samples positively cont…","""29 / 46 samples negatively con…"
"""Os03g0307200""",-180,4,28,46,"""4 / 46 samples positively cont…","""28 / 46 samples negatively con…"


&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;


## Search specific gene related to GO term

In [ ]:
# Search specific gene related to GO:0097577	sequestering of iron ion
specific_gene = ['Os09g0396900', 'Os11g0106700', 'Os12g0106000']
specific_gene_df = HNscore_sample_count.filter(
    pl.col('GENEID').is_in(specific_gene)
)
display(specific_gene_df)

In [ ]:
# Search specific gene related to GO:0006879	intracellular iron ion homeostasis
specific_gene_down = ['Os01g0689451', 'Os02g0649900', 'Os02g0650300', 'Os03g0667300', 'Os05g0551000', 'Os07g0257200', 'Os07g0258400']
specific_gene_down_df = HNscore_sample_count.filter(
    pl.col('GENEID').is_in(specific_gene_down)
).sort(
    by=["HN5(all)", "GENEID"],
    descending=[False, False]
)
display(specific_gene_down_df)